# Notebook details
- Utilize LLM-as-judge to evaluate generation results

# Install and import libraries

In [ ]:
!pip install transformers datasets evaluate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -U bitsandbytes transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 79.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [ ]:
import json
from transformers import pipeline, AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
import torch
import os

# Load Results

In [ ]:
rag_results_path = "/content/drive/MyDrive/Capstone/eval_samples/rag_outputs.json"
no_rag_results_path = "/content/drive/MyDrive/Capstone/eval_samples/no_rag_outputs.json"

In [ ]:
with open(rag_results_path) as f:
    rag_results = json.load(f)

with open(no_rag_results_path) as f:
    no_rag_results = json.load(f)

# Load config and judge model

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False)

In [ ]:
# base_model = "Qwen/Qwen2.5-14B-Instruct-1M"
# model_name = "qwen"

# base_model = "mistralai/Mistral-7B-Instruct-v0.3"
# model_name = "mistral7B"

base_model = "microsoft/Phi-4-mini-instruct"
model_name = "phi4"

In [ ]:
def get_model(base_model, use_token = False):
  if use_token:
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=quant_config,
        device_map={"": 0},
        trust_remote_code = True,
        use_auth_token='auth_token'
      )

  else:
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=quant_config,
        device_map={"": 0},
        trust_remote_code = True,
      )

  return model

In [ ]:
def get_tokenizer(base_model, use_token = False, set_pad_as_eos = True):
  if use_token:
    tokenizer = AutoTokenizer.from_pretrained(
        base_model,
        trust_remote_code=True,
        use_auth_token='auth_token'
    )

  else:
    tokenizer = AutoTokenizer.from_pretrained(
        base_model,
        trust_remote_code=True
    )

  tokenizer.pad_token = tokenizer.eos_token

  return tokenizer

In [ ]:
model = get_model(base_model, use_token = True)
tokenizer = get_tokenizer(base_model, use_token = True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-4-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/54.3k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/3.91M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

# Set judge-model generation parameters

In [ ]:
max_length = 10000

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer = tokenizer, max_length = max_length, trust_remote_code=True)

Device set to use cuda:0


# Evaluation Pipeline

### Evaluation Prompt

In [ ]:
def generate_eval_prompt(eval_text, input, context):
  prompt = f"""
You are an expert evaluator tasked with scoring the quality of AI-generated explanations based on specific criteria.

### Evaluation Task:
Evaluate the following explanation based on the criteria provided below.

### AI Output to Evaluate:
{eval_text}

### Input given to AI:
{input}

### Context given to AI:
{context}

### Evaluation Criteria:
1. **Relevance**: Is the explanation relevant to the task, features and SHAP values provided in the input? Does it avoid irrelevant details?
2. **Correct Identification of Top Features**: Does the output correctly identify the top 10 influential features based on SHAP values?
3. **Correct Explanation of Feature Effects**: Are the explanations for how features affect the model's decision clear and accurate, aligning with domain knowledge?
4. **Contextually Accurate**: Does the explanation adhere to the context provided in the input?

### Instructions:
- Use a score from 1 to 5 for each criterion, where:
  - 1 = Poor
  - 2 = Below Average
  - 3 = Average
  - 4 = Good
  - 5 = Excellent
- Provide reasoning for each score using chain-of-thought reasoning.
- At the end, provide an overall score (average of all criteria).

### Example Output Format:
Criterion 1 (Relevance): Score = X. Reasoning: ...
Criterion 2 (Identification): Score = X. Reasoning: ...
Criterion 3 (Explanation): Score = X. Reasoning: ...
Criterion 4 (Contextual Accuracy): Score = X. Reasoning:...
Overall Score: X."""

  return prompt

In [ ]:
# Helper function to filter and extract results 
def extract_result_components(result):
  sample_idx = result["sample_index"]
  full_output = result["full_output"]
  model_output = result["model_output"]

  output_idx = full_output.find(model_output)
  context_idx = full_output.find("Context:\n")

  input = full_output[:context_idx]
  context = full_output[context_idx:output_idx]

  return {"sample_index": sample_idx,
          "input": input,
          "context": context,
          "output": model_output}

### Evaluation pipeline

In [ ]:
def eval_pipeline(results, pipe):
  eval_results = []
  for sample in results:
    samp_idx = sample["sample_index"]
    print(f"Starting pipeline for sample index {samp_idx}")
    sample_result = extract_result_components(sample)
    eval_prompt = generate_eval_prompt(eval_text = sample_result["output"],
                                       input = sample_result["input"],
                                       context = sample_result["context"])
    print("Components extracted")

    messages = [
        {"role":"system", "content": "You are an expert evaluator with domain knowledge in loan default prediction."},
        {"role": "user", "content": eval_prompt}
      ]
    output = pipe(messages)
    sample_eval = output[0]["generated_text"][2]["content"]
    print("Evaluation completed")

    eval_results.append(
        {"sample_index": sample["sample_index"],
        "evaluation": sample_eval}
    )
    print("Results appended. Process completed for this sample\n\n")

  return eval_results

### Logging results

In [ ]:
def eval_logger(judge_model, result_type, eval_results):
  return {
      "judge_model": judge_model,
      "result_type": result_type,
      "eval_samples": len(eval_results),
      "eval_results": eval_results
  }

In [ ]:
eval_results = eval_pipeline(results = rag_results, pipe = pipe)
rag_eval_log = eval_logger(judge_model = base_model, result_type = "RAG", eval_results = eval_results)

Starting pipeline for sample index 50
Components extracted
Evaluation completed
Results appended. Process completed for this sample


Starting pipeline for sample index 100
Components extracted
Evaluation completed
Results appended. Process completed for this sample


Starting pipeline for sample index 150
Components extracted
Evaluation completed
Results appended. Process completed for this sample


Starting pipeline for sample index 200
Components extracted
Evaluation completed
Results appended. Process completed for this sample


Starting pipeline for sample index 250
Components extracted
Evaluation completed
Results appended. Process completed for this sample


Starting pipeline for sample index 300
Components extracted
Evaluation completed
Results appended. Process completed for this sample


Starting pipeline for sample index 350
Components extracted
Evaluation completed
Results appended. Process completed for this sample


Starting pipeline for sample index 400
Components extrac

In [ ]:
eval_results = eval_pipeline(results = no_rag_results, pipe = pipe)
no_rag_eval_log = eval_logger(judge_model = base_model, result_type = "No RAG", eval_results = eval_results)

Starting pipeline for sample index 50
Components extracted
Evaluation completed
Results appended. Process completed for this sample


Starting pipeline for sample index 100
Components extracted
Evaluation completed
Results appended. Process completed for this sample


Starting pipeline for sample index 150
Components extracted
Evaluation completed
Results appended. Process completed for this sample


Starting pipeline for sample index 200
Components extracted
Evaluation completed
Results appended. Process completed for this sample


Starting pipeline for sample index 250
Components extracted
Evaluation completed
Results appended. Process completed for this sample


Starting pipeline for sample index 300
Components extracted
Evaluation completed
Results appended. Process completed for this sample


Starting pipeline for sample index 350
Components extracted
Evaluation completed
Results appended. Process completed for this sample


Starting pipeline for sample index 400
Components extrac

In [ ]:
eval_path = "/content/drive/MyDrive/Capstone/eval_samples"

In [ ]:
with open(os.path.join(eval_path, f"rag_eval_{model_name}.json"), "w") as f:
  json.dump(rag_eval_log, f, indent = 4)

In [ ]:
with open(os.path.join(eval_path, f"no_rag_eval_{model_name}.json"), "w") as f:
  json.dump(no_rag_eval_log, f, indent = 4)